##dump-blast-alignments

This file takes in a fplot and rplot file and dumps the ref-to-ref matching into a table (translate_file)

In [1]:
%matplotlib inline
%run ipy_setup.py

In [3]:
translate_file = pd.read_csv("/hpc/users/neffr01/jason_new/nucmer_analysis/translate_file.tsv", 
                            sep='\t', header=None)

In [75]:
fplot_file = '/hpc/users/neffr01/jason_new/nucmer_analysis/chr14-jasonref-full.fplot'
rplot_file = '/hpc/users/neffr01/jason_new/nucmer_analysis/chr14-jasonref.rplot'

In [46]:
class FalconReader:
    
    def __init__ (self, fn):
        self.matches = list(self.read_file_to_matches(fn))
    
    def read_file_to_matches(self, fn):
        with open(fplot_file) as f:
            currMatch = None
            last1 = " "
            last2 = " "
            counter = 1
            for l in f:
                if l[0] == "#": # header information
                    if (last1 != " ") & (last2 != " "):
                        start = last2[0:-1]
                        end = last1[0:-1]
                        yield FalconMatch(start, end)
                last2 = last1
                last1 = l
                counter += 1

class FalconMatch:
    
    def __init__ (self, start, end):        
        start_rpos, start_qpos, start_ident = start.split(' ')[0:3]
        end_rpos, end_qpos, end_ident = end.split(' ')[0:3]
        if int(start_qpos) < int(end_qpos):
            self.direction = 'F'
        else:
            self.direction = 'R'
        self.rstart = int(start_rpos)
        self.rend = int(end_rpos)
        self.qstart = int(start_qpos)
        self.qend = int(end_qpos)
        self.identity = float(start_ident)
        self.chrom = None
        
    def __repr__ (self):
        returnline = [self.chrom, self.rstart, self.rend, self.qstart, self.qend, self.identity]
        return ','.join([str(i) for i in returnline])

In [76]:
forward = FalconReader(fplot_file)
#reverse = FalconReader(rplot_file)

In [77]:
start_int = 105500000
end_int = 106886150
match_array = []
for i in forward.matches:
    if (i.rstart > start_int) & (i.rstart < end_int): # start is within the interval
        match = next((v for k,v in translate_file.iterrows() if v[1] < i.qstart+1))
        i.chrom = match[0]
        match_array.append(["chr14", i.rstart, i.rend, i.rend-i.rstart, i.chrom, i.qstart - match[1], i.qend - match[1], i.qend-i.qstart, i.identity])
    elif (i.rend > start_int) & (i.rend < end_int):
        match = next((v for k,v in translate_file.iterrows() if v[1] < i.qstart+1))
        i.chrom = match[0]
        print "chr14", i.rstart, i.rend, i.chrom, i.qstart - match[1], i.qend - match[1], i.qend-i.qstart

chr14 105510010 105510473 463 001196F 891783 892267 484 79.793814433
chr14 105804016 105804391 375 000062F 559142 559511 369 79.8941798942
chr14 105955194 105955962 768 000697F 642004 642767 763 79.9485861183
chr14 105827758 105828507 749 000923F 608336 609088 752 79.9741602067
chr14 105954756 105956099 1343 000054F 2377797 2379137 1340 79.9853372434
chr14 106797502 106798266 764 000526F 2966640 2967393 753 80.2614379085
chr14 106415847 106416134 287 000110F 2695172 2695471 299 80.4560260586
chr14 105501130 105501414 284 001032F 477999 478278 279 80.6228373702
chr14 106856372 106858139 1767 000086F 1571666 1573457 1791 80.6274078151
chr14 105540134 105540739 605 000205F 5361571 5362158 587 81.0543657331
chr14 105504761 105505016 255 000732F 312844 313122 278 81.4285714286
chr14 106091631 106091926 295 000557F 1880092 1880395 303 81.4332247557
chr14 106472472 106472732 260 000003F 13831333 13831591 258 81.4393939394
chr14 105870090 105870861 771 000366F 3983707 3984487 780 81.4720812183

KeyboardInterrupt: 